In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output


# Configure the plotting routines
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Configure the CRUD module
import CRUD
from CRUD import AnimalShelter

import base64

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

image_filename = 'GS-Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# defines default style for the buttons and style when active
blue_button_style = {'background-color': 'blue',
                      'color': 'white', 'font-size':'16px', 'text-align':'center', 'width':'25%'}
clicked_button_style = {'background-color': 'yellow',
                      'color': 'black', 'font-size':'16px', 'text-align':'center', 'width':'25%'}

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Center(html.A(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
                         height = 250, width = 251), href = 'https://www.snhu.edu', target = "_blank")),
    html.Div(className='buttonRow',
        style={'display': 'flex'},
        children=[
            html.Button(id='submit-button-one', n_clicks_timestamp=-1, n_clicks=0,
                        children='Water', style=blue_button_style),
            html.Button(id='submit-button-two', n_clicks_timestamp=-1, n_clicks=0,
                        children='Mountain/Wilderness', style=blue_button_style),
            html.Button(id='submit-button-three', n_clicks_timestamp=-1, n_clicks=0,
                        children='Disaster/Individual Tracking',style=blue_button_style),
            html.Button(id='submit-button-four', n_clicks_timestamp=-1, n_clicks=0,
                        children='All', style =clicked_button_style)
        ]),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns = [],
        selected_rows = [0],
        page_action="native",
        page_current=0,
        page_size=10,
        style_table={'overflowX': 'scroll'}
    ),
    html.Br(),
    html.Hr(),
    
    html.Div(className='row',
        style={'display' : 'flex', 'justify-content':'center'},
             children=[html.Div(id='graph-id',className='col s12 m6'),
                       html.Div(id='map-id', className='col s12 m6')])
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(
    Output('datatable-id', "data"),
    [Input('submit-button-one', 'n_clicks_timestamp'),
     Input('submit-button-two', 'n_clicks_timestamp'),
     Input('submit-button-three', 'n_clicks_timestamp'),
     Input('submit-button-four', 'n_clicks_timestamp')])

# Filters data table results based on rescue type
def onClick(button1, button2, button3, button4):
    df = pd.DataFrame.from_records(shelter.read({}))
    mostRecentRescueType = max(int(button1), int(button2), int(button3), int(button4))
    
    if (mostRecentRescueType != -1):
        if (mostRecentRescueType == int(button1)):
            df = pd.DataFrame.from_records(shelter.read(
                {"breed":{"$in":["Labrador Retriever Mix","Chesa Bay Retr Mix","Newfoundland Mix"]}, 
                 "sex_upon_outcome":"Intact Female","age_upon_outcome_in_weeks":{"$gte":26, "$lte":156}}))
        elif (mostRecentRescueType == int(button2)):
            df = pd.DataFrame.from_records(shelter.read(
                {"breed":{"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog", 
                                "Siberian Husky","Rottweiler"]},
                "sex_upon_outcome":"Intact Male", "age_upon_outcome_in_weeks":{"$gte":26, "$lte":156}}))
        elif (mostRecentRescueType == int(button3)):
            df = pd.DataFrame.from_records(shelter.read(
                {"breed":{"$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound",
                                "Rottweiler"]},
                "sex_upon_outcome":"Intact Male", "age_upon_outcome_in_weeks":{"$gte":20, "$lte":300}}))           
            
    df.drop(columns=['_id'],inplace=True)
    return df.to_dict('records')

    
@app.callback(
    Output('submit-button-one', 'style'),
    Output('submit-button-two', 'style'),
    Output('submit-button-three', 'style'),
    Output('submit-button-four', 'style'),
    [Input('submit-button-one', 'n_clicks_timestamp'),
     Input('submit-button-two', 'n_clicks_timestamp'),
     Input('submit-button-three', 'n_clicks_timestamp'),
     Input('submit-button-four', 'n_clicks_timestamp')])

# updates button styles when clicked/active
def styleClick(button1, button2, button3, button4):
    mostRecentRescueType = max(int(button1), int(button2), int(button3), int(button4))
    bt1 = blue_button_style
    bt2 = blue_button_style
    bt3 = blue_button_style
    bt4 = clicked_button_style
    if (mostRecentRescueType != -1):   
        if (mostRecentRescueType == int(button1)):
            bt1 = clicked_button_style
            bt4 = blue_button_style
        elif (mostRecentRescueType == int(button2)):
            bt2 = clicked_button_style
            bt4 = blue_button_style
        elif (mostRecentRescueType == int(button3)):
            bt3 = clicked_button_style
            bt4 = blue_button_style
            
    return bt1, bt2, bt3, bt4
          
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'} for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):   
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can 
    # be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '500px', 'height': '500px'},
            center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for 
            # the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
                children=[
                    dl.Tooltip(dff.iloc[row,4]),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[row,9] if dff.iloc[row,9] != "" else "No name on record.")
                    ])
                ])
            ])
    ]

# uses plotly to generate a pie chart from the data currently in the table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(dff, names='breed', title="Percentage by Breed")
        )]
    

app.run(jupyter_mode="tab");